In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score

In [3]:
from scipy.stats import pearsonr

In [4]:
%store -r df_Xy

In [5]:
df_Xy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5894 entries, 1997-07-02 to 2020-05-27
Columns: 242 entries, gbp_rate to gold_change%_lag10
dtypes: float64(242)
memory usage: 10.9 MB


# We want to investigate the predictivty of central bank policy rates and stock market index performance as they relate to the price of gold. 
### We have the most recent 10 years of daily data from the following sources:
### Features:
#### Top 5 equity market indices by market capitalization, per Statista: https://www.statista.com/statistics/270126/largest-stock-exchange-operators-by-market-capitalization-of-listed-companies/, and their corresponding daily performance and percent change
NYSE Composite Index history: https://finance.yahoo.com/quote/%5ENYA?p=^NYA&.tsrc=fin-srch
NASDAQ Composite Index history: https://finance.yahoo.com/quote/%5EIXIC?p=^IXIC&.tsrc=fin-srch
Japan Exchange Group Composite Index history: https://finance.yahoo.com/quote/8697.T?p=8697.T&.tsrc=fin-srch
Shanghai Stock Exchange Composite Index history: https://finance.yahoo.com/quote/%5ESSEC?p=^SSEC&.tsrc=fin-srch
Hang Seng (Hong Kong) Index History: https://finance.yahoo.com/quote/%5EHSI?p=^HSI&.tsrc=fin-srch
#### Top 5 Currencies Globally in Forex Trading (https://www.ig.com/us/trading-strategies/top-10-most-traded-currency-pairs-191206), and their corresponding daily central bank policy rates and percent change
The Fed (USD): https://fred.stlouisfed.org/series/FEDFUNDS
ECB (EUR): https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html
BoJ (JPY): https://fred.stlouisfed.org/series/IRSTCI01JPM156N
United Kingdom (GBP): https://fred.stlouisfed.org/series/GBPONTD156N
Australia (AUD): https://www.rba.gov.au/statistics/cash-rate/
#### Daily price of gold  and percent change (up to 10-day lag): https://www.usagold.com/reference/prices/goldhistory.php



### Targets:
#### Daily price of gold, daily percent change price of gold: https://www.usagold.com/reference/prices/goldhistory.php

## As the data sits, we currently have both daily policy rates and index scores - each with up to a ten day lag as the data. This translates to 242 features in our would-be model.

### For sake of dimensionality reduction, let's first get just the raw values into one data frame, and the percent change values into another. It will be interesting to see if gold's price or gold's percent price change is 'easier' to predict. 

In [6]:
# store 'change' column titles to create separate dataframes

col_dif = []
col_raw = []
for col in df_Xy.columns:
    if 'change%' in col:
        col_change.append(col)
    elif 'dif' in col:
        col_dif.append(col)
    else:
        col_raw.append(col)

(['gbp_rate_change',
  'usd_rate_change',
  'eur_rate_change',
  'aud_rate_change',
  'yen_rate_change',
  'gbp_rate_change_lag1',
  'gbp_rate_change_lag2',
  'gbp_rate_change_lag3',
  'gbp_rate_change_lag4',
  'gbp_rate_change_lag5',
  'gbp_rate_change_lag6',
  'gbp_rate_change_lag7',
  'gbp_rate_change_lag8',
  'gbp_rate_change_lag9',
  'gbp_rate_change_lag10',
  'usd_rate_change_lag1',
  'usd_rate_change_lag2',
  'usd_rate_change_lag3',
  'usd_rate_change_lag4',
  'usd_rate_change_lag5',
  'usd_rate_change_lag6',
  'usd_rate_change_lag7',
  'usd_rate_change_lag8',
  'usd_rate_change_lag9',
  'usd_rate_change_lag10',
  'eur_rate_change_lag1',
  'eur_rate_change_lag2',
  'eur_rate_change_lag3',
  'eur_rate_change_lag4',
  'eur_rate_change_lag5',
  'eur_rate_change_lag6',
  'eur_rate_change_lag7',
  'eur_rate_change_lag8',
  'eur_rate_change_lag9',
  'eur_rate_change_lag10',
  'aud_rate_change_lag1',
  'aud_rate_change_lag2',
  'aud_rate_change_lag3',
  'aud_rate_change_lag4',
  'aud_r

In [7]:
# create three separate dataframes, one with raw values, one with differences, and one with percent change

df_Xy_raw = df_Xy[col_raw].dropna()

df_Xy_dif = df_Xy[col_dif].dropna()

df_Xy_change = df_Xy[col_change].dropna()

In [8]:
df_Xy_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3022 entries, 2008-09-19 to 2020-05-27
Columns: 121 entries, gbp_rate to gold_lag10
dtypes: float64(121)
memory usage: 2.8 MB


In [ ]:
df_Xy_df.info()

In [9]:
df_Xy_change.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5894 entries, 1997-07-02 to 2020-05-27
Columns: 121 entries, gbp_rate_change to gold_change%_lag10
dtypes: float64(121)
memory usage: 5.5 MB


#### To compare models on a level playing field, we need to ensure the same date range is being used for both the raw and the change data. Let's take a look at the date ranges in all of our datasets

In [10]:
to_drop = df_Xy_change.index[df_Xy_change.index < '2008-09-19']

df_Xy_change.drop(to_drop, inplace = True)

In [11]:
df_Xy_change.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3022 entries, 2008-09-19 to 2020-05-27
Columns: 121 entries, gbp_rate_change to gold_change%_lag10
dtypes: float64(121)
memory usage: 2.8 MB


In [12]:
# create X,y pairs for both raw and percent change dataframes

X_raw = df_Xy_raw.drop(columns = ['gold'])
y_raw = df_Xy_raw['gold']

X_change = df_Xy_change.drop(columns = ['gold_change%'])
y_change = df_Xy_change['gold_change%']

### Now, to inform our feature selection for our models, let's take a look at the linear relationships between our X features and y targets in both datasets.